In [ ]:
#import packages needed for running this code
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
import xlsxwriter

#import Gurobi solver; Gurobi must be installed separately prior to running this code
from gurobipy import *

In [ ]:
#import the opioid poisoning incidents
incidents = pd.read_csv("Opioid poisonings.csv")

In [ ]:
#import the transit stop locations
stops = pd.read_csv("Transit stops.csv")

In [ ]:
#import the distance matrix between all opioid poisonings and all transit stops
#this was pre-computed using QGIS 3.30
distmat = pd.read_csv("Poisoning-Transit distance table.csv")

In [ ]:
#set coverage distance limit

#coverage_distance = 100
coverage_distance = 160
#coverage_distance = 280

In [ ]:
#define the covering variable stop_adj[i,j] as 1 if an opioid poisoning is covered by a transit stop, and 0 otherwise

stop_adj = {}
    
for row in distmat.itertuples():
        
    if (row.total_cost <= coverage_distance):
        stop_adj[row.incident_id,row.stop_id] = 1
    else: 
        stop_adj[row.incident_id,row.stop_id] = 0

In [ ]:
#define N as an array of values for naloxone kits that we want to consider placing
N = np.arange(10,1010,10)

#set up K-fold cross validation
KF = KFold(n_splits = 5, shuffle = True)
itera = 0

#create and initialize the result summary Excel spreadsheet
ff = xlsxwriter.Workbook('Optimization Result.xlsx')

#create sheet to summarize opioid poisonings covered
sheethh = ff.add_worksheet('Poisonings')
sheethh.write(0,0,'N')
sheethh.write(0,1,'Fold')
sheethh.write(0,2,'Poisonings Covered')
rowhh = 1

#begin cross-validation of incidents by running the below analysis for each train/test fold combination
for train, test in KF.split(incidents):
    itera += 1
    print ('Fold: '+str(itera))

    #split the opioid poisoning dataframe into training and testing sets
    train_set = incidents.iloc[train]
    test_set = incidents.iloc[test]

    #initialize the training fold Gurobi model
    m1 = Model() 
    m1.NumScenarios = len(N)
    
    #initialize the testing fold Gurobi model
    mt = Model()
    mt.NumScenarios = len(N)

    
    #initialize variable dictionaries
    x = {} 
    a = {}
    y = {}
    b = {}

    #training set model: let x[j] = 1 if a naloxone kit is placed at transit stop j, and 0 otherwise
    #testing set model: let a[j] be the value of x[j] for transit stop j
    for j in stops['stop_id']: 
        x[j] = m1.addVar(vtype=GRB.BINARY, name="x%d" % j) 
        a[j] = mt.addVar(vtype=GRB.BINARY, name="a%d" % j)

    #training set model: let y[i] = 1 if opioid poisoning i is covered by at least one placed naloxone kit, and 0 otherwise
    for i in train_set['incident_id']:
        y[i] = m1.addVar(vtype=GRB.BINARY, name="y%d" % i)
    
        #constraint: for each opioid poisoning i, y[i] can only equal 1 if there is at least one placed naloxone kit within covering distance
        m1.addConstr(quicksum(stop_adj[i,j]*x[j] for j in stops['stop_id']) >= y[i], "match")

    #objective function: maximize opioid poisoning coverage in training set
    m1.setObjective(quicksum(y[i] for i in train_set['incident_id']), GRB.MAXIMIZE) 

    #constraint: select exactly N kits (the base case is equal to 0 here)
    budget = m1.addConstr(quicksum(x[j] for j in stops['stop_id']) == 0, "budget")
    
    #multiscenario model: adjust the value of N for each scenario
    for kk in range(len(N)):
        m1.Params.ScenarioNumber = kk
        budget.ScenNRhs = N[kk]


    #run the training set model
    m1.update()                     
    m1.optimize() 

    #force a[j] to equal x[j] based on the result of the training set model for each scenario in N
    for kk in range(len(N)):
        m1.Params.ScenarioNumber = kk
        mt.Params.ScenarioNumber = kk
        
        for jj in stops['stop_id']:
            if x[jj].ScenNX > 0.5:
                a[jj].ScenNLB = 0.5
            else: 
                a[jj].ScenNUB = 0.5
    
    
    #testing set model: let b[i] = 1 if opioid poisoning i is covered by at least one placed naloxone kit, and 0 otherwise 
    for i in test_set['incident_id']:
        b[i] = mt.addVar(vtype=GRB.BINARY, name='c%d' % i)
        mt.addConstr(quicksum(stop_adj[i,j]*a[j] for j in stops['stop_id']) >= b[i], 'match')
        
    #objective function: maximize opioid poisoning coverage in testing set
    mt.setObjective(quicksum(b[i] for i in test_set['incident_id']), GRB.MAXIMIZE)

    
    #run the testing set model
    mt.update()
    mt.optimize()

    
    #output opioid poisoning coverage in Excel file
    for kk in range(len(N)):
        mt.Params.ScenarioNumber = kk
        sheethh.write(rowhh,0,N[kk])
        sheethh.write(rowhh,1,itera)
        sheethh.write(rowhh,2,mt.ScenNObjVal)
        rowhh += 1 
        
#close Excel file
ff.close()